# Transfer Learning

![tranfer_inner_chi](assets/transfer/transfer_inner_chi.jpg)

(Image: kuailexiaorongrong.blog.163.com, https://sg.news.yahoo.com/6-kungfu-moves-movies-wished-194241611.html)

# Topics

- Motivation
- Process
- Workshop

# Transfer Learning

Transfering the weights of a neural network to perform a different task.

# Domain Adaptation

![domain adaptation](assets/transfer/Transfer_learning_and_domain_adaptation.png)

(image: [Wikipedia](https://en.wikipedia.org/wiki/Domain_adaptation))

# Motivation

- Lots of data, time, resources needed to train and tune a neural network from scratch
  - An ImageNet deep neural net can take weeks to train and fine-tune from scratch.
  - Unless you have 256 GPUs, possible to achieve in [1 hour](https://research.fb.com/publications/accurate-large-minibatch-sgd-training-imagenet-in-1-hour/)
- Cheaper, faster way of adapting a neural network by exploiting their generalization properties

# Transfer Learning in Practice

Transfer learning is often featured in the "train your own AI model" services:
  - just upload 5-10 images to train a new model

![custom vision](assets/transfer/custom-vision.png)

(image: https://azure.microsoft.com/en-us/services/cognitive-services/custom-vision-service/)

# Neural Network Layers: General to Specific 

- Bottom/first/earlier layers: general learners
 - Low-level notions of edges, visual shapes

- Top/last/later layers: specific learners
  - High-level features such as eyes, feathers
  
Note: the top/bottom notation is confusing, I'd avoid it

## Example: VGG 16 Filters

![vgg filters](assets/transfer/vgg16_filters_overview.jpg)

https://blog.keras.io/how-convolutional-neural-networks-see-the-world.html

![overview](assets/transfer/Transfer+Learning+Overview.jpg)

(image: [Aghamirzaie & Salomon](http://slideplayer.com/slide/8370683/))

# Process

1. Start with pre-trained network

2. Partition network into:
 - Featurizers: identify which layers to keep
 - Classifiers: identify which layers to replace

3. Re-train classifier layers with new data

4. Unfreeze weights and fine-tune whole network with smaller learning rate

## Freezing and Fine-tuning

![vgg 16 modified](assets/transfer/vgg16_modified.png)

(image: http://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)

## Which layers to re-train?

- Depends on the domain
- Start by re-training the last layers (last full-connected and last convolutional)
  - work backwards if performance is not satisfactory

## Example

![transfer performance](assets/transfer/transfer_performance.png)

(image: http://arxiv.org/abs/1411.1792)

## When and how to fine-tune?

Suppose we have model A, trained on dataset A
Q: How do we apply transfer learning to dataset B to create model B?

|Dataset size|Dataset similarity|Recommendation|
|--|--|--|
|Large|Very different|Train model B  from scratch, initialize weights from model A|
|Large|Similar|OK to fine-tune (less likely to overfit)|
|Small|Very different|Train classifier using the earlier layers (later layers won't help much)|
|Small|Similar|Don't fine-tune (overfitting). Train a linear classifier|

https://cs231n.github.io/transfer-learning/

## Learning Rate

- Training linear classifier: typical learning rate

- Fine-tuning: use smaller learning rate to avoid distorting the (already decent) weights


# Workshop: Transfer Learning 